In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# load raw data
df = pd.read_json('../data/fizzy.json', lines=True, dtype=False)

#start cleaning
df['actualCostEth'] = pd.to_numeric('0.' + df.actualCost.str.split(' ').str[0])
df['actualCostUsd'] = pd.to_numeric(df.actualCost.str.split(' ').str[2].str.slice(2,6,1))
df['blockHeight'] = pd.to_numeric(df.blockHeight)
df['gasLimit'] = pd.to_numeric(df.gasLimit)
df['gasPrice'] = pd.to_numeric('0.' + df.gasPrice.str.split(' ').str[0])
df['gasUsed'] = pd.to_numeric(df.gasUsed.str.split(' ').str[0])
df['timeStamp'] = df.timeStamp.str.extract('\((.*)\)')
df['timeStamp'] = pd.to_datetime(df.timeStamp, format='%b-%d-%Y %I:%M:%S %p +%Z')
df['inputData'] = df.inputData.str.replace('[\\r|\\n]', '')
df['functionCalled'] = df.inputData.str.extract('Function: (.*)\(')
df['flightId'] = df.inputData.str.extract('[addNewInsurance|updateFlightStatus|manualInsuranceResolution].*\[0\]:\s\s(.{64})')
df['flightNo'] = df.flightId.str.decode('hex') \
                            .str.decode('utf-8') \
                            .str.rsplit('.',n=1) \
                            .str.get(0) \
                            .str.replace('.','')
df['departureDate'] = pd.to_datetime(df.flightId.str.decode('hex') \
                                     .str.decode('utf-8') \
                                     .str.rsplit('.',n=1) \
                                     .str.get(1) \
                                     .str.slice(0,10,1), unit='s')
df['limitArrivalTime'] = df.inputData.str.extract('addNewInsurance.*\[1\]:\s\s(.{64})')
df['limitArrivalTime'] = pd.to_datetime(df.limitArrivalTime.str[-12:].apply(lambda x: None if pd.isnull(x) else int(str(x), 16)),unit='ms')
df['premium'] = df.inputData.str.extract('addNewInsurance.*\[2\]:\s\s(.{64})')
df['premium'] = df.premium.apply(lambda x: None if pd.isnull(x) else int(str(x), 16))
df['indemnity'] = df.inputData.str.extract('addNewInsurance.*\[3\]:\s\s(.{64})')
df['indemnity'] = df.indemnity.apply(lambda x: None if pd.isnull(x) else int(str(x), 16))
df['productId'] = df.inputData.str.extract('(?:addNewInsurance.*\[4\]:\s\s|manualInsuranceResolution.*\[2\]:\s\s)(.{64})')
df['actualArrivalTime'] = df.inputData.str.extract('updateFlightStatus.*\[1\]:\s\s(.{64})')
df['actualArrivalTime'] = pd.to_datetime(df.actualArrivalTime.str[-12:].apply(lambda x: None if pd.isnull(x) else int(str(x), 16)),unit='ms')
df['newStatusId'] = df.inputData.str.extract('manualInsuranceResolution.*\[1\]:\s\s(.{64})')
df['value'] = pd.to_numeric(df.value.str.slice(0,1,1))
df['eventName'] = df.eventName.str[0:-2]
df['eventStatus'] = df.eventParam5.apply(lambda x: None if x == "None" else int(str(x),16))
df['eventIndemnity'] = df[df['eventName'] == "InsuranceUpdate"]['eventParam4'].apply(lambda x: None if x == "None" else int(str(x),16))
del df['txReceiptStatus']
del df['inputData']
del df['actualCost']
del df['eventParam1']
del df['eventParam2']
del df['eventParam3']
del df['eventParam4']
del df['eventParam5']

# Informations de bases

In [ ]:
loss = df[df['eventStatus'] == 2]['eventIndemnity'].sum()
premium = df[df['eventName'] == 'InsuranceCreation']['premium'].sum()
nbContrat = df['productId'].nunique()
nbClaim = df[df['eventStatus'] == 2]['eventIndemnity'].count()
medianPremium = df[df['eventName'] == 'InsuranceCreation']['premium'].median()
meanPremium = df[df['eventName'] == 'InsuranceCreation']['premium'].mean()
print("Sinistre : " + str(loss) + "€")
print("Primes : " + str(premium) + "€")
print("S/P : " + str(round((loss/premium)*100,2)) + "%")
print("Nombre de contrat : " + str(nbContrat))
print("Nombre de sinistre : " + str(nbClaim))
print("Prime médiane :" + str(medianPremium) + "€")
print("Prime moyenne :" + str(round(meanPremium,2)) + "€")

In [ ]:
#from matplotlib.pyplot import figure
plt.figure(num=None, figsize=(18, 5), dpi=80, facecolor='w', edgecolor='k')

ts = df[df['eventName'] == 'InsuranceCreation']['timeStamp'].groupby(df.timeStamp.dt.floor('d')).size().reset_index(name='count')
plt.plot(ts['timeStamp'], ts['count'])
plt.title("Souscription à Fizzy au court du temps")
plt.xlabel("date")
plt.ylabel("nb de souscription/jour (log)")
plt.yscale("log")
plt.xticks(rotation='45')
plt.grid(True)
plt.show()

In [ ]:
# Les 10 vols les plus assurés
df[df['functionCalled'] == 'addNewInsurance']['flightNo'].value_counts()[0:10]